# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """
    url: str
    title: str
    body: str
    links: List[str]
    text: str 
    
    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [13]:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    
    def __init__(self, url):
        self.url = url 
        response = requests.get(url, headers=headers)
        self.body = response.content 
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.tile else 'No title found'
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator='\n', strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]
        
    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [14]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'ht

In [29]:
print(ed.get_contents())

Webpage Title:
No title found
Webpage Contents:
Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers and ton

In [28]:
ed.title

'No title found'

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [17]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{ 
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [18]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{ 
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [19]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [20]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/2024/12/21/

In [ ]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [31]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/sesame/csm-1b',
 '/google/gemma-3-27b-it',
 '/Qwen/QwQ-32B',
 '/deepseek-ai/DeepSeek-R1',
 '/SparkAudio/Spark-TTS-0.5B',
 '/models',
 '/spaces/sesame/csm-1b',
 '/spaces/Qwen/QwQ-32B-Demo',
 '/spaces/Wan-AI/Wan2.1',
 '/spaces/ASLP-lab/DiffRhythm',
 '/spaces/Kwai-Kolors/Kolors-Virtual-Try-On',
 '/spaces',
 '/datasets/FreedomIntelligence/medical-o1-reasoning-SFT',
 '/datasets/facebook/natural_reasoning',
 '/datasets/open-r1/codeforces-cots',
 '/datasets/Congliu/Chinese-DeepSeek-R1-Distill-data-110k',
 '/datasets/SmallDoge/SmallThoughts',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/Writer',
 '/docs/transformers'

In [23]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [34]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

def get_all_details(url):
    result = "Landing Page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link['url']).get_contents()
    return result

In [35]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'homepage', 'url': 'https://huggingface.co'}, {'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'models page', 'url': 'https://huggingface.co/models'}, {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'}, {'type': 'spaces page', 'url': 'https://huggingface.co/spaces'}, {'type': 'community discussion', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}
Landing Page:
Webpage Title:
No title found
Webpage Contents:
Hugging Face
Models
Datase

In [36]:
print(get_all_details("https://www.eureka.ai"))

Found links: {'links': [{'type': 'about page', 'url': 'https://www.eureka.ai/company/our-organisation'}, {'type': 'team page', 'url': 'https://www.eureka.ai/company/team'}, {'type': 'corporate structure page', 'url': 'https://www.eureka.ai/company/corporate-structure'}]}
Landing Page:
Webpage Title:
No title found
Webpage Contents:
Product
Industries
Eureka AI Moments
Company
Contact Us
Free Trial
Product
Industries
Eureka AI Moments
Company
Contact Us
Free Trial
Contact Us
Free Trial
Product
Industries
Eureka AI Moments
Company
Actionable
Market & Risk Intelligence from Population Scale Data
Grow your business with the most accurate and granular data from the largest continuous research and advertising panel in any country with 30% - 80% population coverage
Grow your business with the most accurate and granular data from the largest continuous research and advertising panel in any country with 30% - 80% population coverage
Start your free trial
Start your free trial
Learn more about u

In [45]:
#system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
#and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
#Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
 and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
 Include details of company culture, customers and careers/jobs if you have the information."


In [39]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [ ]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

In [40]:
get_brochure_user_prompt("Eureka AI", "https://www.eureka.ai")

Found links: {'links': [{'type': 'company page', 'url': 'https://www.eureka.ai/company/our-organisation'}, {'type': 'team page', 'url': 'https://www.eureka.ai/company/team'}, {'type': 'corporate structure page', 'url': 'https://www.eureka.ai/company/corporate-structure'}]}


'You are looking at a company called: Eureka AI\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding Page:\nWebpage Title:\nNo title found\nWebpage Contents:\nProduct\nIndustries\nEureka AI Moments\nCompany\nContact Us\nFree Trial\nProduct\nIndustries\nEureka AI Moments\nCompany\nContact Us\nFree Trial\nContact Us\nFree Trial\nProduct\nIndustries\nEureka AI Moments\nCompany\nActionable\nMarket & Risk Intelligence from Population Scale Data\nGrow your business with the most accurate and granular data from the largest continuous research and advertising panel in any country with 30% - 80% population coverage\nGrow your business with the most accurate and granular data from the largest continuous research and advertising panel in any country with 30% - 80% population coverage\nStart your free trial\nStart your free trial\nLearn more about us\nLearn more about us\nTrusted by global & local i

In [41]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [ ]:
create_brochure("HuggingFace", "https://huggingface.co")

In [42]:
create_brochure("Eureka AI",  "https://www.eureka.ai")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.eureka.ai/company/our-organisation'}, {'type': 'team page', 'url': 'https://www.eureka.ai/company/team'}, {'type': 'corporate structure page', 'url': 'https://www.eureka.ai/company/corporate-structure'}]}


```markdown
# **Eureka AI Brochure**

## **About Us**
Eureka AI is at the forefront of market and risk intelligence, leveraging the largest continuous research and advertising panel to deliver actionable insights. Our cutting-edge technologies harness privacy-compliant data from 30% - 80% of the population, allowing businesses to thrive in a dynamic marketplace.

## **Our Products**
We offer three integrated products designed to empower your business:

### **1. Omni**
- **Description:** The most accurate and granular insight on the Digital Market.
- **Features:** Provides a thorough understanding of market share and customer profiles using anonymized data.
- **Unique Value:** Utilizes privacy-compliant data covering 30% - 80%+ of the population.
  
### **2. Explorer**
- **Description:** Audience Creation for Targeted Advertising.
- **Features:** Enables the creation of deterministic audiences based on credit score, socio-economic status, and app usage.
- **Unique Value:** Reach target audiences across digital advertising platforms including Meta, Google, and TikTok.

### **3. Spectrum**
- **Description:** Alternative Data Credit Score to Increase Approvals.
- **Features:** A unique credit scoring system that combines alternative data with traditional credit bureau scores.
- **Unique Value:** Allows banks to increase approval rates by 10-20% with maintained levels of non-performing loans.

## **Industries We Serve**
Eureka AI caters to various industries requiring rigorous market intelligence and targeted advertising solutions, including finance, retail, and tech sectors.

## **Eureka AI Moments**
Beyond products, we celebrate moments of success and growth with our clients as they navigate and capitalize on the digital economy.

## **Company Culture**
At Eureka AI, we pride ourselves on fostering an inclusive and innovative workplace culture. Our team members are passionate about using data to drive decisions and deliver real-world impact. Collaboration, continuous learning, and agility are at the core of our values, which translates into exceptional service for our clients.

## **Join Us**
Eureka AI is always looking for passionate individuals ready to make a difference. If you're interested in joining a team that values creativity and data-driven decision-making, explore current job openings on our careers page.

## **Get in Touch**
Ready to harness the power of data for your business? 
- [Start Your Free Trial](#)
- [Contact Us](#) for more information!

Eureka AI is trusted by global and local industry leaders – let us help you grow your business today!
```


## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [43]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [ ]:
stream_brochure("HuggingFace", "https://huggingface.co")

In [44]:
stream_brochure("Eureka AI",  "https://www.eureka.ai")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.eureka.ai/company/our-organisation'}, {'type': 'team page', 'url': 'https://www.eureka.ai/company/team'}, {'type': 'corporate structure page', 'url': 'https://www.eureka.ai/company/corporate-structure'}, {'type': 'contact page', 'url': 'https://www.eureka.ai/contact'}]}



# Eureka AI Brochure

## Welcome to Eureka AI
Eureka AI is at the forefront of market and risk intelligence, utilizing population-scale data to provide actionable insights. Our mission is to empower businesses with the most accurate and granular data-driven strategies, enabling them to thrive in a competitive landscape.

---

## Our Products
We offer three integrated products designed to enhance your business growth:

### 1. **Omni**
- **Overview:** The premier market intelligence platform offering in-depth insights on market share and customer profiles.
- **Unique Feature:** Utilizes privacy-compliant anonymized data that covers 30% - 80% of the population, ensuring accuracy and relevance.

### 2. **Explorer**
- **Overview:** A powerful audience creation tool for targeted advertising.
- **Unique Feature:** Facilitates the creation of deterministic audiences based on diverse criteria, including credit score and app usage, across prominent digital advertising platforms (e.g., Meta, Google, TikTok).

### 3. **Spectrum**
- **Overview:** An alternative data credit score designed to increase approval rates for financial institutions.
- **Unique Feature:** Combines alternative data with traditional credit bureau scores, resulting in a 10-20% increase in approval rates while maintaining acceptable levels of non-performing loans.

---

## Industries We Serve
Eureka AI caters to both global and local industry leaders across various sectors, including but not limited to finance, advertising, and market research. With our comprehensive data solutions, businesses can navigate complexities and make informed decisions.

---

## Company Culture
At Eureka AI, we foster a culture of innovation, collaboration, and inclusivity. Our team thrives on open communication, diverse perspectives, and a shared commitment to excellence. We encourage continuous learning and provide opportunities for professional growth, ensuring every member can contribute to our mission dynamically.

---

## Careers at Eureka AI
Join us in revolutionizing market intelligence! We are on the lookout for forward-thinking individuals passionate about data and the impact it can have on businesses worldwide. Explore exciting career opportunities across various departments such as Data Science, Marketing, and Sales.

---

## Contact Us
Have a question or want to learn more? Reach out to our team, and kickstart your journey with Eureka AI today!

- **Website:** [Eureka AI](#)
- **Email:** contact@eurekaai.com 
- **Phone:** (123) 456-7890

---

## Try Us for Free!
Experience our products with a free trial. Discover how Eureka AI can transform your business with accurate data insights.

---

*Eureka AI: Transforming Data into Actionable Intelligence*



In [46]:
stream_brochure("Eureka AI",  "https://www.eureka.ai")

Found links: {'links': [{'type': 'company page', 'url': 'https://www.eureka.ai/company/our-organisation'}, {'type': 'team page', 'url': 'https://www.eureka.ai/company/team'}, {'type': 'corporate structure page', 'url': 'https://www.eureka.ai/company/corporate-structure'}]}



# Eureka AI: The Secret Sauce for Business Growth!

Welcome to Eureka AI, where we sprinkle some data magic on your business decisions! 🎩✨

## Our Grand Products
We have three fantastic products that'll make your business thrive faster than you can say "data-driven decisions"! 

- **Omni**: Your digital market's GPS. Get the *most accurate* and *granular* insights on market share and customer profiles. As privacy-compliant as a ninja in the night, Omni covers 30% - 80% of the population. Yes, that means you get the best info without snooping!
  
- **Explorer**: Let’s take a journey into the wild world of advertising! Create audiences so specific they’ll feel like your best bud at the bar. With clever targeting like credit scores, income levels, and app usage, you can find your crowd across platforms like Meta, Google, and TikTok! Don’t worry, we’ll keep your audience's hats on!
  
- **Spectrum**: Meet your new BFF in credit scoring! Spectrum gives banks a magical lift, increasing approvals by 10-20% while maintaining the same levels of non-performing loans. It’s a data combo so good it might make you want to high-five your banker!

## The Eureka AI Experience
Join the league of global and local industry leaders who trust us. When we say *trust us*, we don’t mean the kind of trust you give to your roommate not to eat your leftover pizza, we're talking big leagues here!

## Company Culture
We believe in a culture that celebrates curiosity, innovation, and maybe a little quirkiness! "Eureka!" moments happen here—all while wearing comfy socks and sipping coffee. Don’t be surprised if you catch a team member dancing during a brainstorming session. It’s all part of the job!

## Career Opportunities
Looking to join our team? We're always in search of curious minds who can turn numbers into narratives. If you can outsmart spreadsheets and have a flair for data storytelling, we want YOU! Bring your A-game and a sense of humor, because at Eureka AI, we take fun *very* seriously.

## Get a Taste of Eureka AI
Before you hop on the bandwagon, why not start your free trial? It’s like taste-testing that mysterious dish at an international food fair. You might discover a new favorite!

### Contact Us
Got questions? Pondering whether to call or email us? Just do it! We promise our inbox isn't on fire, and our phones aren’t scary monsters lurking in the dark.

---

So, whether you’re looking to grow your business, join a vibrant culture, or be part of data brilliance, **Eureka AI** is the place for you. Let's make magic happen! 🎉



In [ ]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>